In [1]:
import tensorflow as tf
import numpy as np

In [99]:
#ALTERNATIVE
def parse_record(record):
    name_to_features = {
        'dimZ': tf.io.FixedLenFeature([], tf.int64),
        'dimX': tf.io.FixedLenFeature([], tf.int64),
        'dimY': tf.io.FixedLenFeature([], tf.int64),
        'channels': tf.io.FixedLenFeature([], tf.int64),
        'label': tf.io.FixedLenFeature([], tf.string),
        'feature': tf.io.FixedLenFeature([], tf.string),
    }
    return tf.io.parse_single_example(record, name_to_features)

def decode_record(record):
    feature = tf.io.decode_raw(
        record['feature'], out_type='int16', little_endian=True, fixed_length=None, name=None
    )
    label = tf.io.decode_raw(
        record['label'], out_type='int16', little_endian=True, fixed_length=None, name=None
    )
    dimZ = record['dimZ']
    dimX = record['dimX']
    dimY = record['dimY']
    channels = record['channels']
    print(dimZ, dimX, dimY, channels)
    label = tf.reshape(label, (dimZ, dimX, dimY, channels))
    feature = tf.reshape(feature, (dimZ, dimX, dimY, channels))
    #label.set_shape(dimZ, dimX, dimY, channels)
    #feature.set_shape(dimZ, dimX, dimY, channels)
    return (feature, label)

In [137]:
def parse_record(record):
    name_to_features = {
        'shape': tf.io.FixedLenFeature([], tf.string),
        'label': tf.io.FixedLenFeature([], tf.string),
        'feature': tf.io.FixedLenFeature([], tf.string),
    }
    return tf.io.parse_single_example(record, name_to_features)

def decode_record(record):
    feature = tf.io.decode_raw(
        record['feature'], out_type='int16', little_endian=True, fixed_length=None, name=None
    )
    label = tf.io.decode_raw(
        record['label'], out_type='int16', little_endian=True, fixed_length=None, name=None
    )
    shape = tf.io.decode_raw(
        record['shape'], out_type='int64', little_endian=True, fixed_length=None, name=None
    )

    label = tf.reshape(label, shape)
    feature = tf.reshape(feature, shape)
    label.set_shape((64,256,256,1))
    feature.set_shape((64,256,256,1))
    return (feature, label)

def reset_shape(x, y, shape):
    print(y)
    x.set_shape((None,None,None,None))
    y.set_shape((None,None,None,None))
    return x, y

In [142]:
def printme(x,y):
    print('FEATURE', tf.reduce_min(x), tf.reduce_max(x))
    print('LABEL', tf.reduce_min(y), tf.reduce_max(y))
    return x, y

In [138]:
#ds_path = '/content/drive/MyDrive/RADLogix/dataset'
ds_path = '/Users/dutking/LOCAL/AI_uni/radlogix/dataset/tfrecords'
train_ds_file = 'train.tfrecords'
val_ds_file = 'val.tfrecords'
test_ds_file = 'test.tfrecords'

with tf.device("CPU"):
  train_ds = tf.data.TFRecordDataset(f"{ds_path}/{train_ds_file}", compression_type="GZIP").map(parse_record).map(decode_record)#.map(printme)

 
  
  #val_ds = tf.data.TFRecordDataset(f"{ds_path}/{val_ds_file}", compression_type="GZIP").map(parse_record, num_parallel_calls=tf.data.AUTOTUNE).map(decode_record, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(4, reshuffle_each_iteration = True).batch(1).prefetch(tf.data.AUTOTUNE)
  
  #test_ds = tf.data.TFRecordDataset(f"{ds_path}/{test_ds_file}", compression_type="GZIP").map(parse_record, num_parallel_calls=tf.data.AUTOTUNE).map(decode_record, num_parallel_calls=tf.data.AUTOTUNE).cache().shuffle(4, reshuffle_each_iteration = True).batch(1).prefetch(tf.data.AUTOTUNE)

In [143]:
for x,y in train_ds.take(1):
    printme(x,y)

FEATURE tf.Tensor(-960, shape=(), dtype=int16) tf.Tensor(50, shape=(), dtype=int16)
LABEL tf.Tensor(0, shape=(), dtype=int16) tf.Tensor(1, shape=(), dtype=int16)


In [139]:
norm_ds = train_ds.map(lambda x, y: x)
normalization_layer = tf.keras.layers.Normalization(axis=None)
normalization_layer.adapt(norm_ds)

In [140]:
norm_train_ds = train_ds.map(lambda x,y: (normalization_layer(x), y))

In [144]:
for x,y in norm_train_ds.take(1):
    printme(x,y)

FEATURE tf.Tensor(-2.6901324, shape=(), dtype=float32) tf.Tensor(0.5712873, shape=(), dtype=float32)
LABEL tf.Tensor(0, shape=(), dtype=int16) tf.Tensor(1, shape=(), dtype=int16)
